In [1]:
%pip install openai num2words matplotlib plotly scipy scikit-learn pandas tiktoken redis langchain

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/f1/1f/3a0cb7d172f451b2ca8bf65d9196aa3b6878c010d461257c621e4bd48cad/openai-0.27.10-py3-none-any.whl.metadata
  Using cached openai-0.27.10-py3-none-any.whl.metadata (13 kB)
  Using cached num2words-0.5.12-py3-none-any.whl (125 kB)
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/c9/46/6cbaf20f5bd0e7c1d204b45b853c2cd317b303fada90245f2825ecca47de/matplotlib-3.7.2-cp39-cp39-win_amd64.whl.metadata
  Using cached matplotlib-3.7.2-cp39-cp39-win_amd64.whl.metadata (5.8 kB)
  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/26/5d/1e13b597ed8e54803e9ac6ded18c04cd35d8cbc49016778ec50c4ca9e9d5/plotly-5.16.1-py2.py3-none-any.whl.metadata
  Using cached plotly-5.16.1-py2.py3-none-any.whl.metadata (7.0 kB)
  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/e9/20/2d0561ab54d857365926c5b5353

In [1]:
import openai
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding
import tiktoken
from typing import List
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.redis import Redis as RedisVectorStore

{
  "capabilities": {
    "fine_tune": false,
    "inference": true,
    "completion": false,
    "chat_completion": false,
    "embeddings": true
  },
  "lifecycle_status": "preview",
  "deprecation": {
    "inference": 1738454400
  },
  "id": "text-embedding-ada-002",
  "status": "succeeded",
  "created_at": 1675296000,
  "updated_at": 1675296000,
  "object": "model"
}


In [3]:
# Load data into dataframe

df=pd.read_csv(os.path.join(os.getcwd(),'wiki_movie_plots_deduped.csv'))
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
...,...,...,...,...,...,...,...,...
34881,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ..."
34882,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the..."
34883,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i..."
34884,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...


In [4]:
# Filter data

df.insert(0, 'id', range(0, len(df)))
df['year'] = df['Release Year'].astype(int)
df['origin'] = df['Origin/Ethnicity'].astype(str)
del df['Release Year']
del df['Origin/Ethnicity']
df = df[df.year > 1970] # only movies made after 1970
df = df[df.origin.isin(['American','British','Canadian'])] # only movies from English-speaking cinema
df

,id,Title,Director,Cast,Genre,Wiki Page,Plot,year,origin
8626,8626,$ aka Dollars,Richard Brooks,"Warren Beatty, Goldie Hawn",unknown,https://en.wikipedia.org/wiki/$_(film),"Set in Hamburg, West Germany, several criminal...",1971,American
8627,8627,200 Motels,"Tony Palmer, Charles Swenson","Frank Zappa, Ringo Starr, Theodore Bikel",unknown,https://en.wikipedia.org/wiki/200_Motels,"In 200 Motels, the film attempts to portray th...",1971,American
8628,8628,The Anderson Tapes,Sidney Lumet,"Sean Connery, Dyan Cannon, Christopher Walken,...",unknown,https://en.wikipedia.org/wiki/The_Anderson_Tapes,"Burglar John ""Duke"" Anderson (Sean Connery) is...",1971,American
8629,8629,The Andromeda Strain,Robert Wise,"Arthur Hill, James Olson, Kate Reid, David Way...",unknown,https://en.wikipedia.org/wiki/The_Andromeda_St...,"After a satellite, a U.S. government project c...",1971,American
8630,8630,Bad Man's River,Eugenio Martin,"Lee Van Cleef, Gina Lollobrigida",unknown,https://en.wikipedia.org/wiki/Bad_Man%27s_River,Roy King's gang robs a bank and flees to Mexic...,1971,American
...,...,...,...,...,...,...,...,...,...
22428,22428,"Hochelaga, Land of Souls (Hochelaga terre des ...",François Girard,"Raoul Max Trujillo, Tanaya Beatty, David La Haye",historical drama,"https://en.wikipedia.org/wiki/Hochelaga,_Land_...","One night on the campus of McGill University, ...",2017,Canadian
22429,22429,Indian Horse,Stephen Campanelli,"Forrest Goodluck, Michiel Huisman, Michael Mur...",drama,https://en.wikipedia.org/wiki/Indian_Horse_(film),"The Indian Horse family, including six-year-ol...",2017,Canadian
22430,22430,The Little Girl Who Was Too Fond of Matches (L...,Simon Lavoie,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Girl_...,"In rural 1930s Quebec, Alice lives in house wi...",2017,Canadian
22431,22431,Meditation Park,Mina Shum,"Sandra Oh, Liane Balaban, Don McKellar",drama,https://en.wikipedia.org/wiki/Meditation_Park,"Opened by Mandarin theme song, Meditation Park...",2017,Canadian


In [5]:
# clean text

pd.options.mode.chained_assignment = None

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df['Plot']= df['Plot'].apply(lambda x : normalize_text(x))

In [6]:
#Check how many tokens it will require to encode all the plots

tokenizer = tiktoken.get_encoding("cl100k_base")
df['n_tokens'] = df["Plot"].apply(lambda x: len(tokenizer.encode(x)))
df = df[df.n_tokens<8192]
print('Number of movies: ' + str(len(df))) # print number of movies remaining in dataset
print('Number of tokens required:' + str(df['n_tokens'].sum())) # print number of tokens

Number of movies: 11125
Number of tokens required:7044844


In [8]:
# load everything into Langchain

from langchain.document_loaders import DataFrameLoader
loader = DataFrameLoader(df, page_content_column="Plot" )
movie_list = loader.load()

In [9]:
# create embeddings and load redis

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.redis import Redis as RedisVectorStore


# we will use OpenAI as our embeddings provider
embedding = OpenAIEmbeddings(
    deployment="<your-deployment-name>", # your name for the model deployment (e.g. "my-ml-model")
    model="<your-deployment-model>", # name of the specific ML model (e.g. "text-embedding-ada-002" )
    openai_api_base="<your-azureopenai-endpoint>",
    openai_api_type="azure", # you need this if you're using Azure Open AI service
    openai_api_key="<your-azureopenai-key>",
    openai_api_version="2023-05-15",
    chunk_size=16 # current limit with Azure OpenAI service. This will likely increase in the future.
    )

# name of the Redis search index to create
index_name = "movieindex"

REDIS_ENDPOINT = "<your-endpoint>" # must include port at the end. e.g. "redisdemo.eastus.redisenterprise.cache.azure.net:10000"
REDIS_PASSWORD = "<your-password>"

# create a connection string for the Redis Vector Store. Uses Redis-py format: https://redis-py.readthedocs.io/en/stable/connections.html#redis.Redis.from_url
# This example assumes TLS is enabled. If not, use "redis://" instead of "rediss://
redis_url = "rediss://:" + REDIS_PASSWORD + "@"+ REDIS_ENDPOINT

# create and load redis with documents
vectorstore = RedisVectorStore.from_documents(
    documents=movie_list,
    embedding=embedding,
    index_name=index_name,
    redis_url=redis_url
)

# This may take up to 10 minutes to complete. 

In [12]:
vectorstore.write_schema("redis_schema.yaml")
# this saves a copy of the vectorstore schema so you can load it later without having to re-create the vectorstore

In [21]:
# Basic similarity query

results = vectorstore.similarity_search_with_score("dogs playing basketball")
print('Title: '+ str(results[0][0].metadata['Title']))
print('Vector Distance: ' + str(results[0][1]))

Title: BASEketball
Vector Distance: 0.1859


In [22]:
# Hybrid similarity query with a genre filter first

from langchain.vectorstores.redis import RedisText, RedisNum, RedisTag

genre_filter = RedisText("Genre") == "family"
results = vectorstore.similarity_search_with_score("dogs playing basketball", filter=genre_filter)
topanswer = results
print('Title: '+ str(results[0][0].metadata['Title']))
print('Vector Distance: ' + str(results[0][1]))

Title: Air Bud
Vector Distance: 0.1907
